In [1]:
import pandas as pd
import tensorflow as tf
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
import optuna

In [2]:
data = pd.read_csv('./sentences.csv')
data.head()

,sentence_number,word,tag
0,0,O,OTHER
1,0,que,OTHER
2,0,aconteceu,OTHER
3,0,nos,OTHER
4,0,últimos,C-TIME


In [3]:
vocab = set(data['word'])
vocab_list = list(vocab)

word2idx = {}
idx2word = {}

for idx, word in enumerate(vocab_list):
    word2idx[word] = idx
    idx2word[idx] = word

tags = set(data['tag'])
tags_list = list(tags)

tag2idx = {}
idx2tag = {}

for idx, tag in enumerate(tags_list):
    tag2idx[tag] = idx
    idx2tag[idx] = tag

In [4]:
converted_data = data.copy()

converted_data['word'] = converted_data['word'].transform(lambda word: word2idx[word])
converted_data['tag'] = converted_data['tag'].transform(lambda tag: tag2idx[tag])
converted_data.head()

,sentence_number,word,tag
0,0,23,12
1,0,26,12
2,0,12,12
3,0,0,12
4,0,9,2


In [5]:
list_data = converted_data.groupby(['sentence_number'])['word', 'tag'].agg(lambda i: list(i))
list_data.head()

<ipython-input-5-1584793e4124>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  list_data = converted_data.groupby(['sentence_number'])['word', 'tag'].agg(lambda i: list(i))


,word,tag
sentence_number,,
0,"[23, 26, 12, 0, 9, 13, 26, 29, 34, 30, 18]","[12, 12, 12, 12, 2, 2, 12, 12, 7, 12, 3]"
1,"[4, 16, 29, 25, 36, 1, 33, 2, 27, 24]","[12, 12, 12, 12, 12, 8, 12, 7, 0, 11]"
2,"[7, 20, 14, 10, 5]","[12, 10, 12, 12, 4]"
3,"[6, 37, 19, 8, 21, 28]","[12, 6, 12, 10, 12, 1]"
4,"[7, 29, 15, 22, 17, 31]","[12, 12, 5, 7, 12, 7]"


In [6]:
max_len = list_data['word'].map(len).max()
max_len

11

In [7]:
pad_tokens = tf.keras.preprocessing.sequence.pad_sequences(list_data['word'])
pad_tags = tf.keras.preprocessing.sequence.pad_sequences(list_data['tag'])
pad_tags = [tf.keras.utils.to_categorical(tag, num_classes=len(tags)) for tag in pad_tags]

train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags)

In [8]:
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

In [9]:
input_dim = len(vocab)
input_length = max_len

def objective(trial):
    embedding_output_dim = trial.suggest_int("embedding_output_dim", 16, 256)

    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=embedding_output_dim, input_length=input_length),
        Bidirectional(LSTM(units=embedding_output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'),
        LSTM(units=embedding_output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),
        TimeDistributed(Dense(len(tags), activation='relu'))
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    num_epochs = trial.suggest_int("num_epochs", 5, 150)

    model.fit(train_tokens, np.array(train_tags), verbose=1, epochs=num_epochs)

    return model.evaluate(test_tokens, np.array(test_tags), return_dict=True)['accuracy']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-08-01 20:25:48,981] A new study created in memory with name: no-name-5c6439f6-8ff3-4b11-883b-5e555c6bff16


Epoch 1/91
1/1 [==============================] - 10s 10s/step - loss: 6.0082 - accuracy: 0.0682
Epoch 2/91
1/1 [==============================] - 0s 358ms/step - loss: 3.2190 - accuracy: 0.4318
Epoch 3/91
1/1 [==============================] - 0s 458ms/step - loss: 3.1495 - accuracy: 0.4545
Epoch 4/91
1/1 [==============================] - 0s 403ms/step - loss: 3.0294 - accuracy: 0.4545
Epoch 5/91
1/1 [==============================] - 0s 446ms/step - loss: 3.0090 - accuracy: 0.4773
Epoch 6/91
1/1 [==============================] - 0s 420ms/step - loss: 2.9952 - accuracy: 0.4545
Epoch 7/91
1/1 [==============================] - 0s 432ms/step - loss: 2.9730 - accuracy: 0.5227
Epoch 8/91
1/1 [==============================] - 0s 448ms/step - loss: 2.9495 - accuracy: 0.5000
Epoch 9/91
1/1 [==============================] - 0s 389ms/step - loss: 2.9101 - accuracy: 0.4773
Epoch 10/91
1/1 [==============================] - 0s 404ms/step - loss: 2.8987 - accuracy: 0.5000
Epoch 11/91
1/1 [===

[I 2021-08-01 20:26:32,286] Trial 0 finished with value: 0.5 and parameters: {'embedding_output_dim': 218, 'num_epochs': 91}. Best is trial 0 with value: 0.5.


Epoch 1/11
1/1 [==============================] - 11s 11s/step - loss: 9.7169 - accuracy: 0.0909
Epoch 2/11
1/1 [==============================] - 0s 345ms/step - loss: 6.2516 - accuracy: 0.4318
Epoch 3/11
1/1 [==============================] - 0s 327ms/step - loss: 5.8819 - accuracy: 0.4318
Epoch 4/11
1/1 [==============================] - 0s 274ms/step - loss: 6.1306 - accuracy: 0.4318
Epoch 5/11
1/1 [==============================] - 0s 409ms/step - loss: 5.3418 - accuracy: 0.4318
Epoch 6/11
1/1 [==============================] - 0s 341ms/step - loss: 2.3152 - accuracy: 0.4318
Epoch 7/11
1/1 [==============================] - 0s 254ms/step - loss: 2.2182 - accuracy: 0.4318
Epoch 8/11
1/1 [==============================] - 0s 247ms/step - loss: 2.1955 - accuracy: 0.4318
Epoch 9/11
1/1 [==============================] - 0s 203ms/step - loss: 2.1947 - accuracy: 0.4318
Epoch 10/11
1/1 [==============================] - 0s 278ms/step - loss: 2.2400 - accuracy: 0.3636
Epoch 11/11
1/1 [===

[I 2021-08-01 20:26:47,661] Trial 1 finished with value: 0.5 and parameters: {'embedding_output_dim': 179, 'num_epochs': 11}. Best is trial 0 with value: 0.5.


...

[I 2021-08-01 21:07:52,971] Trial 97 finished with value: 0.6818181872367859 and parameters: {'embedding_output_dim': 52, 'num_epochs': 71}. Best is trial 2 with value: 0.8181818127632141.


Epoch 1/34
1/1 [==============================] - 9s 9s/step - loss: 7.9263 - accuracy: 0.2500
Epoch 2/34
1/1 [==============================] - 0s 190ms/step - loss: 2.7702 - accuracy: 0.4773
Epoch 3/34
1/1 [==============================] - 0s 176ms/step - loss: 2.3640 - accuracy: 0.4545
Epoch 4/34
1/1 [==============================] - 0s 194ms/step - loss: 2.5581 - accuracy: 0.4545
Epoch 5/34
1/1 [==============================] - 0s 208ms/step - loss: 2.1575 - accuracy: 0.4545
Epoch 6/34
1/1 [==============================] - 0s 178ms/step - loss: 1.5806 - accuracy: 0.4318
Epoch 7/34
1/1 [==============================] - 0s 191ms/step - loss: 1.9269 - accuracy: 0.4318
Epoch 8/34
1/1 [==============================] - 0s 190ms/step - loss: 1.9013 - accuracy: 0.4545
Epoch 9/34
1/1 [==============================] - 0s 239ms/step - loss: 2.0370 - accuracy: 0.3636
Epoch 10/34
1/1 [==============================] - 0s 182ms/step - loss: 1.9572 - accuracy: 0.3409
Epoch 11/34
1/1 [=====

[I 2021-08-01 21:08:10,808] Trial 98 finished with value: 0.0 and parameters: {'embedding_output_dim': 136, 'num_epochs': 34}. Best is trial 2 with value: 0.8181818127632141.


Epoch 1/55
1/1 [==============================] - 9s 9s/step - loss: 8.3565 - accuracy: 0.0227
Epoch 2/55
1/1 [==============================] - 0s 142ms/step - loss: 3.3699 - accuracy: 0.5227
Epoch 3/55
1/1 [==============================] - 0s 167ms/step - loss: 2.6923 - accuracy: 0.5000
Epoch 4/55
1/1 [==============================] - 0s 167ms/step - loss: 2.4881 - accuracy: 0.5909
Epoch 5/55
1/1 [==============================] - 0s 153ms/step - loss: 2.5123 - accuracy: 0.5227
Epoch 6/55
1/1 [==============================] - 0s 177ms/step - loss: 2.4663 - accuracy: 0.5909
Epoch 7/55
1/1 [==============================] - 0s 181ms/step - loss: 2.4477 - accuracy: 0.5455
Epoch 8/55
1/1 [==============================] - 0s 185ms/step - loss: 2.4059 - accuracy: 0.5909
Epoch 9/55
1/1 [==============================] - 0s 225ms/step - loss: 2.3652 - accuracy: 0.5909
Epoch 10/55
1/1 [==============================] - 0s 171ms/step - loss: 2.3351 - accuracy: 0.6364
Epoch 11/55
1/1 [=====

[I 2021-08-01 21:08:30,098] Trial 99 finished with value: 0.5 and parameters: {'embedding_output_dim': 83, 'num_epochs': 55}. Best is trial 2 with value: 0.8181818127632141.


In [ ]:
t = 'Como conseguir dinheiro'.split(' ')
t2 = [word2idx[word] for word in t]
t3 = model.predict(t2)
[idx2tag[np.argmax(cat)] for cat in t3]